Classifying HS Code with Naive Bayes
===

In [28]:
import pandas as pd
import numpy as np
from time import time

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report

#urls = 'https://github.com/dragon-library/work_space/raw/main/HS_Code/HS/hs_code.xlsx'
urls = 'data/hs_code.xlsx'
types = 'section'
#types = "chapter"

def get_master(sheets,types = 'section'):
    data = pd.read_excel(urls,sheet_name= sheets)
    data[types] = data[types].map('{:02}'.format)
    data = data[[types,'description']]
    data['description'] = data['description'].str.lower()
 #   data = data.rename(columns={'heading' : 'target', 'product_desc' : 'question_text'})
     

    return data

def manage_data(df):
    df.columns =   ['topic', 'body']  
    
    
    return df

In [4]:
types = "section"
#types = "chapter"

print("Load the dataset: Section")
t0 = time()

sheets = '8_digit'
eights = get_master(sheets,types)
sheets = '6_digit'
sixs = get_master(sheets,types)
sheets = '4_digit'
fours = get_master(sheets,types)
sheets = '2_digit'
twos = get_master(sheets,types)

sheets = 'test_01'
tests = get_master(sheets,types)

sheets = 'Declaration_2019_10'
decl = get_master(sheets,types)

data = pd.concat([eights,sixs,fours,twos,tests,decl], ignore_index=True)
docs = manage_data(data)


#twenty_test = manage_data(tests)
load_time = time() - t0
print("Load dataset time:  %0.3fs" % load_time)
docs.head()

Load the dataset: Section
Load dataset time:  204.698s


NameError: name 'docs' is not defined

In [30]:
docs.columns =   ['topic', 'body']  
docs

,topic,body
0,01,"horses; live, purebred breeding animals - pure..."
1,01,"horses; live, other than purebred breeding ani..."
2,01,asses; live - other
3,01,mules and hinnies; live- other
4,01,"cattle; live, purebred breeding animals - pure..."
...,...,...
42552,15,adjust gear set jfkz658b
42553,15,aluminium window frame kf057072avat
42554,07,acrylic plate 4mm. size24x24 cm.
42555,16,ac power cord p/n 141102240p6


In [31]:
y = pd.factorize(docs.topic)[0]
X = docs.body
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

In [32]:
vectorizer = CountVectorizer()
X_train_dtm = vectorizer.fit_transform(X_train)
X_test_dtm = vectorizer.transform(X_test)
X_train_dtm.shape, X_test_dtm.shape

((29789, 26066), (12768, 26066))

In [33]:
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)
accuracy_score(y_test, y_pred_class)

0.8594141604010025

In [34]:
pd.DataFrame(confusion_matrix(y_true=y_test, y_pred=y_pred_class))

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,377,0,0,9,0,3,1,1,0,0,...,0,1,0,0,1,0,0,0,0,0
1,3,302,0,14,0,6,1,0,0,0,...,0,0,0,2,1,0,0,0,0,0
2,3,0,67,3,0,3,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,8,2,0,551,0,10,2,0,1,0,...,0,0,1,1,1,2,0,0,1,0
4,1,1,0,0,92,27,0,0,0,0,...,0,1,0,12,6,1,0,0,0,0
5,9,5,1,16,3,1033,12,0,1,0,...,0,4,0,74,31,8,0,0,2,0
6,0,0,0,9,0,20,516,0,0,2,...,0,3,0,44,58,115,1,0,2,0
7,0,0,0,1,0,3,1,84,0,0,...,2,0,0,4,8,4,0,0,2,0
8,1,0,0,1,0,2,2,1,158,0,...,0,0,0,6,4,1,0,0,1,0
9,0,0,0,1,0,7,6,1,0,181,...,0,1,0,5,6,4,1,0,0,0


In [35]:
print (classification_report(y_test,  y_pred_class))

              precision    recall  f1-score   support

           0       0.93      0.96      0.94       394
           1       0.96      0.91      0.93       332
           2       0.99      0.87      0.92        77
           3       0.88      0.95      0.91       582
           4       0.95      0.64      0.77       143
           5       0.90      0.86      0.88      1205
           6       0.87      0.66      0.75       785
           7       0.93      0.71      0.81       118
           8       0.98      0.88      0.93       179
           9       0.95      0.84      0.89       216
          10       0.91      0.96      0.93       928
          11       0.95      0.72      0.82        98
          12       0.92      0.74      0.82       241
          13       0.97      0.68      0.80       156
          14       0.87      0.92      0.89      2909
          15       0.83      0.86      0.84      2262
          16       0.72      0.91      0.81      1486
          17       0.96    

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [42]:
from tradingview_ta import TA_Handler
handler = TA_Handler()

# Tesla's stock
handler.set_symbol_as("TSLA")
handler.set_interval_as(Interval.INTERVAL_1_DAY)
analysis = handler.get_analysis()
print(analysis.summary)

NameError: name 'Interval' is not defined

In [43]:
from tradingview_ta import TA_Handler, Interval

tesla = TA_Handler()
tesla.set_symbol_as("TSLA")
tesla.set_exchange_as_crypto_or_stock("NASDAQ")
tesla.set_screener_as_stock("america")
tesla.set_interval_as(Interval.INTERVAL_1_DAY)
print(tesla.get_analysis().summary)

{'RECOMMENDATION': 'SELL', 'BUY': 6, 'SELL': 13, 'NEUTRAL': 9}
